In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract
import io
import PyPDF2
import sys

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path

import util
import lines
import group
import label
import records
import extract

In [2]:
test_files_dir = ".././test_files/"
test_dfs_dir = test_files_dir + "tesseract_data_frames/"
test_output_dir = test_files_dir + "extracted_indexes/"
test_files = util.list_files(test_files_dir)
test_dfs = util.list_files(test_dfs_dir)
file = test_dfs[2]

indexes_path = "H:\Documents\Indexex\ILO_yearly_indices\\tesseract_data_frames"
indexes_csv = util.list_files(indexes_path, suffix=".csv")
output_dir = "H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes"

display(test_dfs, file)

['.././test_files/tesseract_data_frames/\\LS_index_1920.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1921.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1944.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1976.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1986.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1986_split.csv']

'.././test_files/tesseract_data_frames/\\LS_index_1944.csv'

In [3]:
#pdf_df = util.ocr(file, start_page=1, save_to=".././test_files")
pdf_df = pd.read_csv(file)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)

lines_df

,line_text,artifact_text,x0,y0,x1,y1,page,dx
0,INTERNATIONAL LABOUR OFFICE,,144,342,1899,405,1,1755
1,"LEGISLATIVE SERIES, 1944",,137,1363,1897,1512,1,1760
2,INDEXES,,808,2129,1232,2180,1,424
3,"MONTREAL, CANADA",,719,3137,1314,3181,1,595
4,1946,,967,3207,1072,3248,1,105
...,...,...,...,...,...,...,...,...
4162,Apr. 28 L.D. respecting the organisation of la...,,345,2947,2056,2991,82,1711
4163,agreements; labour courts]. No. 311. (Zb. pp. ...,,608,2993,1881,3039,82,1273
4164,G.O. = Gaceta Oficial de los Estados Unidos de...,,351,3081,1649,3126,82,1298
4165,"Zb. = Zbornik sakona 1 naredaba,",,345,3138,1087,3177,82,742


In [4]:
#rec_df = extract.extract_indexes(pdf_df, save_to=test_output_dir + os.path.basename(file))
rec_df = extract.extract_indexes(pdf_df)
#rec_df.to_csv("LS_index_1944.csv")
rec_df

,country,region,text,page
0,ARGENTINE REPUBLIC,,"June 3 Decree No. 13,943/44, to repeal section...",3
1,ARGENTINE REPUBLIC,,"Sept. 11 Decrees No. 24,335/44 and 24,336/44, ...",3
2,ARGENTINE REPUBLIC,,"Nov. 4 Decree No. 29,830/44, to amend certain ...",3
3,AUSTRALIA,,"Apr. 6 An Act to,amend the Invalid and Old-age...",3
4,AUSTRALIA,,Dec. 21 Amendment of the National Security (In...,3
...,...,...,...,...
1315,URUGUAY,,"Nov.:30 ‘Decree No. 283, fo setup a National C...",82
1316,URUGUAY,,"j . of Social Services. (G.O,: 4th December, 1...",82
1317,URUGUAY,,Dec. 28 Mines Act [repeals Mines Act of 7th Au...,82
1318,URUGUAY,,"Dec, 28-,, ‘Decree No. 305: 1 to promulgate th...",82


In [7]:
rec_df.loc[rec_df["page"]==11]

,country,region,text,page
112,SWITZERLAND,,June 9 Resolution of the Federal Council to re...,11
113,SWITZERLAND,,June 9 Resolution of the Federal Council to am...,11
114,SWITZERLAND,,June 9 Resolution of the Federal Council to am...,11
115,SWITZERLAND,,Oct. 10 Resolutions of the Federal Council to ...,11
116,SWITZERLAND,,Dec. 22 Resolution of the Federal Council to r...,11
117,UNION OF SOUTH AFRICA,,May 30 Act to amend... the Native Service Cont...,11
118,UNION OF SOUTH AFRICA,,June 6 Act to amend ... the Old Age Pensions A...,11
119,NORTHERN IRELAND,,July 28 National Health Insurance (Emergency) ...,11
120,NORTHERN IRELAND,,"Dec. 21 Expiring Laws Continuance Act, 1944 [s...",11
121,NORTHERN IRELAND,Northern Ireland,June 6 An Act to amend certain provisions of t...,11


In [15]:
def group_lines(df, by):
    #d = 4 old value without ocr
    d = 20
    last = "last_" + by
    last_m = last + "_mean"
    bins = pd.DataFrame(columns=[by, "lines", last, last_m, "count"])

    for index, row in df.iterrows():
        x0 = row[by]
        poss_bin = bins.loc[(bins[last_m]-d <= x0) & (bins[last_m]+d >= x0)]

        if poss_bin.empty:
            new_row = pd.DataFrame({
                by: [x0],
                "lines": [index],
                last: [x0],
                last_m: [x0],
                "count": 1
            })
            bins = pd.concat([new_row, bins.loc[:]]).reset_index(drop=True)
        else:
            b = bins.iloc[poss_bin.index[-1]]

            if not type(b[0]) is list:
                b[0] = [b[0]]
                b[1] = [b[1]]
                b[2] = [b[2]]

            b[by].append(x0)
            b["lines"].append(index)
            b["count"] += 1
            
            if len(b[last]) >= 2:
                b[last].pop(0)
            b[last].append(x0)
            
            last_mean = b[last]
            if type(last_mean) is list:
                last_mean = sum(b[last]) / len(b[last])
            b[last_m] = last_mean
            

    return bins

In [16]:
def get_relevant_x0_bins(bins_x0, x0_n, drop_first=False):

    bins_x0_rel = pd.DataFrame(columns=bins_x0.columns)
    for p_no, p in bins_x0.groupby("page"):
        x = p.sort_values(by="last_x0_mean")

        if drop_first:
            x = x.drop(x.iloc[0].name)

        bins_x0_rel = pd.concat([bins_x0_rel, x.iloc[0:1]]) # add lines that start by the left text border

        z = x.drop(x.iloc[0].name)
        z = z.sort_values(by="count", ascending=False).iloc[0:x0_n-1].sort_values(by="last_x0_mean")
        bins_x0_rel = pd.concat([bins_x0_rel, z]) # add the other x0_n-1 bins

    return bins_x0_rel

In [17]:
reg_art = "^[\W_]*([oeau]{2,})?\s?[\W_]*(?<!\()"
re.search(reg_art, ".. 11th k")

<re.Match object; span=(0, 3), match='.. '>

In [18]:
lines_df.loc[lines_df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941
324,NORWAY (continued),,92,400,692,455,9,600
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438
330,PANAMA,,89,796,353,840,9,264
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450


In [19]:
d = lines_df.loc[lines_df["page"]==9]
d_x0 = group_lines(d, "x0")
d_x0["page"] = 9
d_x0

,x0,lines,last_x0,last_x0_mean,count,page
0,152,370,152,152,1,9
1,413,346,413,413,1,9
2,"[391, 389, 385, 391, 391, 389, 389, 392, 391, ...","[328, 329, 332, 333, 334, 337, 338, 340, 341, ...","[393, 389]",391.0,23,9
3,"[137, 136, 134, 134, 136, 137, 149, 138, 138, ...","[325, 327, 336, 339, 345, 350, 354, 357, 360, ...","[120, 136]",128.0,11,9
4,"[92, 112, 89, 117, 90, 92, 90, 104, 94, 94, 108]","[324, 326, 330, 331, 335, 351, 353, 365, 366, ...","[94, 108]",101.0,11,9
5,898,323,898,898,1,9


In [20]:
r_x0 = get_relevant_x0_bins(d_x0, 3)
r_x0

,x0,lines,last_x0,last_x0_mean,count,page
4,"[92, 112, 89, 117, 90, 92, 90, 104, 94, 94, 108]","[324, 326, 330, 331, 335, 351, 353, 365, 366, ...","[94, 108]",101.0,11,9
3,"[137, 136, 134, 134, 136, 137, 149, 138, 138, ...","[325, 327, 336, 339, 345, 350, 354, 357, 360, ...","[120, 136]",128.0,11,9
2,"[391, 389, 385, 391, 391, 389, 389, 392, 391, ...","[328, 329, 332, 333, 334, 337, 338, 340, 341, ...","[393, 389]",391.0,23,9


In [21]:
lines_df.loc[r_x0.iloc[0]["lines"]]

,line_text,artifact_text,x0,y0,x1,y1,page,dx
324,NORWAY (continued),,92,400,692,455,9,600
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551
330,PANAMA,,89,796,353,840,9,264
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716
335,PORTUGAL,,90,1115,421,1169,9,331
351,"Tes ‘of 25th May, 1931, to set up the National...",,92,2092,1831,2160,9,1739
353,SWEDEN,,90,2248,356,2305,9,266
365,L.S. 1936 (Nor. 4); amendments: 1937 (Nor. 5);...,*,104,3008,1837,3093,9,1733
366,1930 (Nor. 2); amendments: 1932 (Nor. 4); 1933...,,94,3065,1834,3132,9,1740
367,(Nor. 2); 1937 (Nor. 1); 1941 (Nor. 1); 1942 (...,,94,3116,1699,3180,9,1605


In [7]:
df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
df = label.assign_labels(df, x0_n)

co_df, p_l, p_g = label.correct_x0_types(df, bins_x0, bins_x1, x0_n)
ind_df = label.assign_labels(co_df, x0_n)
ind_df = label.approve_correction(df, ind_df, p_l)

ind_df["new_label"] = ind_df["label"]
ind_df = label.improve_country_classification(ind_df)

ind_df = records.extract_records(ind_df)

In [22]:
df.loc[df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941,-1,2,other
324,NORWAY (continued),,92,400,692,455,9,600,0,0,country
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698,1,2,start
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551,0,1,country
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698,1,2,start
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442,2,2,middle
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438,2,2,middle
330,PANAMA,,89,796,353,840,9,264,0,0,country
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716,0,2,other
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450,2,2,middle


In [23]:
co_df.loc[co_df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941,-1,2,other
324,NORWAY (continued),,92,400,692,455,9,600,1,0,country
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698,2,2,start
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551,1,1,country
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698,2,2,start
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442,3,2,middle
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438,3,2,middle
330,PANAMA,,89,796,353,840,9,264,1,0,country
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716,1,2,other
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450,3,2,middle


In [26]:
co, p_l, p_g = correct_x0_types(df, bins_x0, bins_x1, x0_n)

Width median 1752.2894736842106
[(1658.9333333333332, 2), (1775.093023255814, 14), (1703.0959302325582, 15), (1709.0063795853268, 25), (1707.2626538987688, 39), (1828.5263157894738, 64), (1705.3108974358975, 69), (1726.8848484848486, 70), (1703.9268292682927, 77), (1728.189024390244, 78), (1702.4452690166975, 79), (1732.2894736842106, 81)]
Pages small width [2, 15, 25, 39, 69, 70, 77, 78, 79, 81]
Pages big width [14, 64]


In [28]:
co.loc[co["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941,-1,2,other
324,NORWAY (continued),,92,400,692,455,9,600,0,0,country
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698,1,2,start
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551,0,1,country
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698,1,2,start
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442,2,2,middle
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438,2,2,middle
330,PANAMA,,89,796,353,840,9,264,0,0,country
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716,0,2,other
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450,2,2,middle


In [9]:
ind_df.loc[ind_df["page"]==10]

,country,region,text,page
90,SWEDEN,,"on _ workers’ protection™, . (S.F,-p, 259)» - ...",10
91,SWEDEN,,"1931, respecting sick funds® <r (S.F. pp. 444-...",10
92,SWEDEN,,SWITZERLAND CONFEDERATION Jan. 400 Resolution ...,10
93,SWEDEN,,"1935 (Swe. 4); 1936 (Swe. 7, A); see also: 193...",10
94,SWEDEN,,"(Swe. 2, C). * Consolidated text: L.S. 1931 (S...",10
95,SWEDEN,,"(Swe. 2); 1937 (Swe. 3); 1938 (Swe. 4). — *""L....",10
96,SWEDEN,,"1 S, 1930 (Swe. 1); amendments: L.S. 1938 (Swe...",10
97,SWEDEN,,"1942 (Swe. 2, B). — “LS. 1940 (Swe. 1, A). 1S....",10
98,SWEDEN,,Bulletin of the International Labour Office (B...,10
99,SWEDEN,,"269-283, and Vol. XIV, 1919, pp. 205-209. — “L...",10


In [25]:
# correct x0_type for pages where something went wrong
def correct_x0_types(lines_df, bins_x0, bins_x1, x0_n):
    df = lines_df.copy()
    start_page = lines_df["page"].min()

    text_widths = list(lines.make_borders_df(bins_x0, bins_x1)["dx"]) # difference between mean of first and last bin for x0 for every page

    tw = text_widths.copy()
    tw.sort()
    width_median = tw[int(len(tw)/2)]
    print("Width median", width_median)

    f = filter(lambda w: (not util.similar_to(w, width_median, 20)), text_widths) # filter widths that differ from the rest
    f = list(f)

    p = []
    for w in range(len(f)):
        p.append((f[w], text_widths.index(f[w]) + start_page)) # add page to strange width
    
    print(p)

    p_l = [a for w, a in p if w<width_median]
    p_g = [a for w, a in p if w>width_median]
    
    print("Pages small width", p_l)
    print("Pages big width", p_g)

    df.loc[df["page"].isin(p_l) & (df["x0_type"]>=0), "x0_type"] +=1 # correct wrong x0_type for p_l

    # correct wrong x0_type (and x1_type) for p_g
    bins = group.get_line_start_end_bins(df.loc[df["page"].isin(p_g)])
    bins_x0_cor = group.get_relevant_x0_bins(bins[0], x0_n, drop_first=True) # drop bin on the very left
    df_cor = label.assign_types(df.loc[df["page"].isin(p_g)], bins_x0_cor, bins_x1.loc[bins_x1["page"].isin(p_g)], x0_n)
    df.loc[df["page"].isin(p_g), ["x0_type", "x1_type"]] = df_cor[["x0_type", "x1_type"]]

    return df, p_l, p_g

In [17]:
for i, file in enumerate(indexes_csv):
    save_path = output_dir + os.sep + os.path.basename(file)
    extract.extract_indexes(pd.read_csv(file), save_to=save_path)

Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1976.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1977.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1978.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1979.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1981.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1982.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_Index_1983.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1983_Spanish.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1984.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_year

## Probleme
- Dokumente mit 2 Spalten FIXED
- Dokumente mit Länder-Überschrift über ganze Breite der Seite IMPROVED
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944
        - S. 8-9, schlechte Ländererkennung FIXED
            - S. 25 falsches Labeling FIXED
        - S. 28 Inhalt fehlt? TESSERACT PROBLEM
        - S. 10-11, schlechte Ländererkennung
    

## Fragen

